# System of Linear Equations

### Import Libraries

In [1]:
import numpy
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# define n*m array by accepting user input and printing it
def create_array(n, m):
    arr = []
    for i in range(n):
        row = list(map(float, input(f"Enter row {i+1} (space-separated {m} values): ").split()))
        while len(row) != m:
            print(f"Please enter exactly {m} values.")
            row = list(map(float, input(f"Enter row {i+1} (space-separated {m} values): ").split()))
        arr.append(row)
    return np.array(arr)

In [3]:
# rows = int(input("Enter the number of equations (rows): "))
# cols = int(input("Enter the number of variables (columns): "))
#print(f"Creating a {rows}x{cols} matrix.")
#matrix = create_array(rows,cols)
matrix = create_array(4,4)

In [4]:
# Forward Elimination
def forward_elimination(matrix, arr_b_u):
    n = matrix.shape[0]
    for k in range(n-1):
        for i in range(k+1, n):
            m = matrix[i][k] / matrix[k][k]
            for j in range(k, n):
                matrix[i][j] -= m * matrix[k][j]
            arr_b_u[i] -= m * arr_b_u[k]
    #print(f"Matrix after forward elimination: {arr_b_u}")
    return matrix, arr_b_u

In [5]:
# Create a blank array for constants
arr_b = np.zeros(matrix.shape[0])

# Create an array for x values with dimension of no of columns in matrix
arr_x = np.zeros(matrix.shape[1])

In [6]:
# Fill the array with user input
arr_b = list(map(float, input(f"Enter the constants array (space-separated {matrix.shape[0]} values): ").split()))
while len(arr_b) != matrix.shape[0]:
    print(f"Please enter exactly {matrix.shape[0]} values.")
    arr_b = list(map(float, input(f"Enter the constants array (space-separated {matrix.shape[0]} values): ").split()))
arr_b = np.array(arr_b)

In [7]:
# Back Substitution
def back_substitution(matrix_c, b_arr, x_arr):
    n = matrix_c.shape[0]
    x_arr[n-1] = b_arr[n-1] / matrix_c[n-1][n-1]
    for i in range(n-2, -1, -1):
        sum_ax = 0
        for j in range(i+1, n):
            sum_ax += matrix_c[i][j] * x_arr[j]
        x_arr[i] = (b_arr[i] - sum_ax) / matrix_c[i][i]
    return x_arr

In [8]:
# Solving the system of equations
print("Original Matrix:\n", matrix)
arr_b_Transpose = arr_b.reshape(-1, 1)
print("Constants array (as column vector):\n", arr_b_Transpose)
# print("Constants array:", arr_b)
print("X values array:", arr_x)
upper_matrix, arr_b_changed = forward_elimination(matrix.copy(), arr_b.copy())
print("Upper Triangular Matrix:\n", upper_matrix)
arr_b_changed_Transpose = arr_b_changed.reshape(-1, 1)
print("Modified Constants array (as column vector):\n", arr_b_changed_Transpose)
#print("Modified Constants array:", arr_b_changed)
arr_x = back_substitution(upper_matrix.copy(), arr_b_changed.copy(), arr_x)
arr_x_Transpose = arr_x.reshape(-1, 1)
print("Solution (X values):", arr_x_Transpose)

Original Matrix:
 [[ 4. -1.  0.  0.]
 [-1.  4. -1.  0.]
 [ 0. -1.  4. -1.]
 [ 0.  0. -1.  3.]]
Constants array (as column vector):
 [[ 5.]
 [ 5.]
 [10.]
 [ 5.]]
X values array: [0. 0. 0. 0.]
Upper Triangular Matrix:
 [[ 4.         -1.          0.          0.        ]
 [ 0.          3.75       -1.          0.        ]
 [ 0.          0.          3.73333333 -1.        ]
 [ 0.          0.          0.          2.73214286]]
Modified Constants array (as column vector):
 [[ 5.        ]
 [ 6.25      ]
 [11.66666667]
 [ 8.125     ]]
Solution (X values): [[1.92810458]
 [2.7124183 ]
 [3.92156863]
 [2.97385621]]


In [92]:
# Checking the solution
def inverse_matrix(matrix_c):
    n = matrix_c.shape[0]
    identity = np.eye(n)
    augmented = np.hstack((matrix_c, identity))

    for i in range(n):
        # Make the diagonal contain all 1's
        diag = augmented[i][i]
        for j in range(2*n):
            augmented[i][j] /= diag

        # Make the other rows contain 0's in the current column
        for k in range(n):
            if k != i:
                factor = augmented[k][i]
                for j in range(2*n):
                    augmented[k][j] -= factor * augmented[i][j]

    inverse = augmented[:, n:]
    return inverse

In [94]:
arr_x_check = np.dot(inverse_matrix(matrix), arr_b)
print("Checked Solution (X values):", arr_x_check)

Checked Solution (X values): [4.         0.28571429 0.42857143 0.71428571]


## Jacobi Iterative Method

In [28]:
def Jacobi_Iterative_method(A, b, tolerance = 10e-3):
    jac_x_old = [b[i]/A[i][i] for i in range(A.shape[0])]
    print("Jacobi Iterative method:")
    print("Initial Guess")
    print(np.array(jac_x_old).reshape(-1, 1))
    jac_x_new = [0]*A.shape[0]
    res = [0]*A.shape[0]
    max_iter = 100
    for _ in range(max_iter):
        for i in range(A.shape[0]):
            sum_m = 0
            for j in range(A.shape[1]):
                sum_m += A[i][j] * jac_x_old[j]
            sum_m -= A[i][i] * jac_x_old[i]
            jac_x_new[i] = (b[i] - sum_m)/A[i][i]
            res[i] = abs(jac_x_new[i] - jac_x_old[i])
        jac_x_old = jac_x_new
        print("Iteration #", _+1)
        print(np.array(jac_x_old).reshape(-1, 1))
        print("Residue at #", _+1)
        print(np.array(res).reshape(-1, 1))
        if max(res) < tolerance:
            print("Converged")
            return jac_x_new
    print("No converged")
    return jac_x_new

In [29]:
print("Original Matrix:\n", matrix)
arr_b_Transpose = arr_b.reshape(-1, 1)
print("Constants array (as column vector):\n", arr_b_Transpose)
jacobi_x = Jacobi_Iterative_method(matrix, arr_b)
print("Jacobi Result : ", np.array(jacobi_x).reshape(-1, 1))
print("Solution (X values):", arr_x_Transpose)
print("difference: ", arr_x_Transpose - np.array(jacobi_x).reshape(-1, 1))

Original Matrix:
 [[ 4. -1.  0.  0.]
 [-1.  4. -1.  0.]
 [ 0. -1.  4. -1.]
 [ 0.  0. -1.  3.]]
Constants array (as column vector):
 [[ 5.]
 [ 5.]
 [10.]
 [ 5.]]
Jacobi Iterative method:
Initial Guess
[[1.25      ]
 [1.25      ]
 [2.5       ]
 [1.66666667]]
Iteration # 1
[[1.5625    ]
 [2.1875    ]
 [3.22916667]
 [2.5       ]]
Residue at # 1
[[0.3125    ]
 [0.9375    ]
 [0.72916667]
 [0.83333333]]
Iteration # 2
[[1.796875  ]
 [2.50651042]
 [3.7516276 ]
 [2.9172092 ]]
Residue at # 2
[[0.]
 [0.]
 [0.]
 [0.]]
Converged
Jacobi Result :  [[1.796875  ]
 [2.50651042]
 [3.7516276 ]
 [2.9172092 ]]
Solution (X values): [[1.92810458]
 [2.7124183 ]
 [3.92156863]
 [2.97385621]]
difference:  [[0.13122958]
 [0.20590788]
 [0.16994102]
 [0.05664701]]


## Gauss-Seidel Iteration

In [35]:
def GS_Iterative_method(A, b, tolerance=1e-5, max_iter=100):
    """
    Solves the system of linear equations Ax = b using the Gauss-Seidel iterative method.

    Args:
        A (np.ndarray): The coefficient matrix (n x n). Must be diagonally dominant.
        b (np.ndarray): The constant vector (n x 1).
        tolerance (float): The desired tolerance for convergence.
        max_iter (int): The maximum number of iterations to perform.

    Returns:
        np.ndarray: The solution vector x, or None if it does not converge.
    """
    # Ensure A and b are NumPy arrays for efficient calculations
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)

    n = A.shape[0]

    # Initialize the solution vector x, typically with zeros
    x = np.zeros_like(b, dtype=float)

    print("Gauss-Seidel Iterative Method")
    print("----------------------------")
    print(f"Initial Guess (x_0):\n{x.reshape(-1, 1)}\n")

    # --- Main Iteration Loop ---
    for k in range(max_iter):
        # Store the old x vector to check for convergence later
        x_old = x.copy()

        # --- Inner loop to update each component of x ---
        for i in range(n):
            # Calculate the sum of A[i, j] * x[j] for j != i
            # np.dot is efficient for this dot product
            sigma = np.dot(A[i, :i], x[:i]) + np.dot(A[i, i+1:], x_old[i+1:])

            # Update the i-th component of the solution vector
            x[i] = (b[i] - sigma) / A[i, i]

        print(f"Iteration #{k+1}")
        print(f"{x.reshape(-1, 1)}\n")

        # --- Convergence Check ---
        # Calculate the infinity norm of the difference between the new and old solution
        # This checks if the solution has stabilized.
        diff_norm = np.linalg.norm(x - x_old, ord=np.inf)

        # So this 'if' statement works perfectly!
        if diff_norm < tolerance:
            print(f"🎉 Converged after {k+1} iterations!")
            return x

    # If the loop finishes without converging
    print(f"Did not converge within {max_iter} iterations.")
    return x


In [36]:
print("Original Matrix:\n", matrix)
arr_b_Transpose = arr_b.reshape(-1, 1)
print("Constants array (as column vector):\n", arr_b_Transpose)
GS_x = GS_Iterative_method(matrix, arr_b)
print("Jacobi Result : ", np.array(GS_x).reshape(-1, 1))
print("Solution (X values):", arr_x_Transpose)
print("difference: ", arr_x_Transpose - np.array(GS_x).reshape(-1, 1))

Original Matrix:
 [[ 4. -1.  0.  0.]
 [-1.  4. -1.  0.]
 [ 0. -1.  4. -1.]
 [ 0.  0. -1.  3.]]
Constants array (as column vector):
 [[ 5.]
 [ 5.]
 [10.]
 [ 5.]]
Gauss-Seidel Iterative Method
----------------------------
Initial Guess (x_0):
[[0.]
 [0.]
 [0.]
 [0.]]

Iteration #1
[[1.25      ]
 [1.5625    ]
 [2.890625  ]
 [2.63020833]]

Iteration #2
[[1.640625  ]
 [2.3828125 ]
 [3.75325521]
 [2.91775174]]

Iteration #3
[[1.84570312]
 [2.64973958]
 [3.89187283]
 [2.96395761]]

Iteration #4
[[1.9124349 ]
 [2.70107693]
 [3.91625864]
 [2.97208621]]

Iteration #5
[[1.92526923]
 [2.71038197]
 [3.92061704]
 [2.97353901]]

Iteration #6
[[1.92759549]
 [2.71205313]
 [3.92139804]
 [2.97379935]]

Iteration #7
[[1.92801328]
 [2.71235283]
 [3.92153804]
 [2.97384601]]

Iteration #8
[[1.92808821]
 [2.71240656]
 [3.92156314]
 [2.97385438]]

Iteration #9
[[1.92810164]
 [2.7124162 ]
 [3.92156764]
 [2.97385588]]

Iteration #10
[[1.92810405]
 [2.71241792]
 [3.92156845]
 [2.97385615]]

🎉 Converged after 10 i